In [2]:
!mkdir data/ 
!wget https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar -O data/ILSVRC2012_img_val.tar
!tar -xf data/ILSVRC2012_img_val.tar -C data/
!wget https://raw.githubusercontent.com/Evolving-AI-Lab/ppgn/master/misc/map_clsloc.txt
!wget https://raw.githubusercontent.com/yqihao/PTMValidations/main/imagenet_classes.txt
!wget https://raw.githubusercontent.com/yqihao/PTMValidations/main/ILSVRC2012_validation_ground_truth.txt

--2022-05-01 17:16:16--  https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar
Resolving image-net.org (image-net.org)... 171.64.68.16
Connecting to image-net.org (image-net.org)|171.64.68.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6744924160 (6.3G) [application/x-tar]
Saving to: ‘data/ILSVRC2012_img_val.tar’

data/ILSVRC2012_img 100%[===================>]   6.28G  38.8MB/s    in 3m 51s  

2022-05-01 17:20:07 (27.8 MB/s) - ‘data/ILSVRC2012_img_val.tar’ saved [6744924160/6744924160]

--2022-05-01 17:21:00--  https://raw.githubusercontent.com/Evolving-AI-Lab/ppgn/master/misc/map_clsloc.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24366 (24K) [text/plain]
Saving to: ‘map_clsloc.txt’

map_clsloc.txt      100%[====

In [4]:
from torchvision import models
import torch
from PIL import Image

In [12]:
target_model = models.efficientnet_b2(pretrained=True)
target_model.eval()

Downloading: "https://download.pytorch.org/models/efficientnet_b2_rwightman-bcdf34b7.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b2_rwightman-bcdf34b7.pth


  0%|          | 0.00/35.2M [00:00<?, ?B/s]

EfficientNet(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): ConvNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): ConvNormActivation(
 

In [13]:
from torchvision import transforms
transform = transforms.Compose([            #[1]
 transforms.Resize(256),                    #[2]
 transforms.CenterCrop(224),                #[3]
 transforms.ToTensor(),                     #[4]
 transforms.Normalize(                      #[5]
 mean=[0.485, 0.456, 0.406],                #[6]
 std=[0.229, 0.224, 0.225]                  #[7]
 )])

In [14]:
with open('imagenet_classes.txt') as f:
  classes = [line.strip() for line in f.readlines()]

mapping_file = open('map_clsloc.txt',mode='r')

validation_file = open('ILSVRC2012_validation_ground_truth.txt',mode='r')
validation_list = validation_file.readlines()
mapping_list_from = mapping_file.readlines()
mapping_list_to = []
classes_mapping_list = []
classes_mapping = {}
for i in range(len(mapping_list_from)):
  classes_mapping[mapping_list_from[i].split(" ")[2].replace('\n',"")] = mapping_list_from[i].split(" ")[1]
for i in range(len(classes)):
  classes_mapping_list.append(classes_mapping[classes[i].split(", ")[1]])

for i in range(len(validation_list)):
  validation_list[i] = validation_list[i].split('\n')[0]

In [15]:
correct = 0
top_5 = 0

it = 50000

for i in range(it):
  img_path = f'data/ILSVRC2012_val_{(i+1):08d}.JPEG'
  # print(img_path)
  # img = Image.open("dog.jpg")
  img = Image.open(img_path).convert('RGB')
  img_t = transform(img)
  batch_t = torch.unsqueeze(img_t, 0)

  out = target_model(batch_t)

  _, index = torch.topk(input=out,k=5)

  # print(index[0].tolist())

  if classes_mapping_list[index[0][0]] == validation_list[i]:
    correct = correct + 1
    top_5 = top_5 + 1
  else:
    for ii in range(1,5):
      if classes_mapping_list[index[0][ii]] == validation_list[i]:
        top_5 = top_5 + 1
        break
  # print(correct, top_5)
  if i!=0 and i%500==0:
    print(str(int(i/500))+'%, Top 1:' + str(correct/i) + ' Top 5:' + str(top_5/i))

accuracy = correct/it*100
top_5_accuracy = top_5/it*100
print('Top 1 Accuracy:'+str(accuracy)+'%')
print('Top 5 Accuracy:'+str(top_5_accuracy)+'%')

1%, Top 1:0.782 Top 5:0.952
2%, Top 1:0.779 Top 5:0.945
3%, Top 1:0.7933333333333333 Top 5:0.944
4%, Top 1:0.7825 Top 5:0.938
5%, Top 1:0.7884 Top 5:0.9404
6%, Top 1:0.7843333333333333 Top 5:0.9376666666666666
7%, Top 1:0.7825714285714286 Top 5:0.9388571428571428
8%, Top 1:0.78 Top 5:0.937
9%, Top 1:0.7811111111111111 Top 5:0.9357777777777778
10%, Top 1:0.7806 Top 5:0.934
11%, Top 1:0.7818181818181819 Top 5:0.9341818181818182
12%, Top 1:0.7825 Top 5:0.9355
13%, Top 1:0.7832307692307693 Top 5:0.9361538461538461
14%, Top 1:0.7835714285714286 Top 5:0.9357142857142857
15%, Top 1:0.7828 Top 5:0.9358666666666666
16%, Top 1:0.781125 Top 5:0.935625
17%, Top 1:0.7817647058823529 Top 5:0.9356470588235294
18%, Top 1:0.7814444444444445 Top 5:0.9357777777777778
19%, Top 1:0.7825263157894737 Top 5:0.9363157894736842
20%, Top 1:0.7819 Top 5:0.9354
21%, Top 1:0.7819047619047619 Top 5:0.9351428571428572
22%, Top 1:0.7813636363636364 Top 5:0.9351818181818182
23%, Top 1:0.780608695652174 Top 5:0.93530434